set up

In [1]:
import os
import sys
import json
import psycopg2
import configparser

In [2]:
is_notebook = True
notebook_prefix = ""
if is_notebook: notebook_prefix = "."

In [3]:
# Load config file
config = configparser.ConfigParser()
config.read(notebook_prefix + './config/config.conf')

# Load configuration file (json format) for source database 
with open(notebook_prefix + config['path']['config.sv']) as json_file:
    source = json.load(json_file)

DATA_DIR = config['path']['source']
HOST = config['database']['host.sv']
PORT = config['database']['port.sv']


path = config['path']
temp_dir = notebook_prefix + f".{path['temp']}"

sys.path.append(config['path']['root'])

from utils import (insert as insert, parse as parse, process, validate, loggr)

## Transform

In [5]:
from importlib import reload
reload(validate)
reload(loggr)
reload(parse)
reload(process)
reload(insert)


# Connect to Postgres database
cnx = psycopg2.connect(
    database=source['database'],
    user=source['user'],
    password=source['password'],
    host=source.get('host', 'localhost'),  # 'localhost' as default
    port=source.get('port', '5432')  # '5432' is postgres' default
)


# Usage
temp_dir = notebook_prefix + f"{path['temp']}"
for filename in os.listdir(temp_dir):
    print(filename)
    if filename.endswith('.txt'):
        full_path = os.path.join(temp_dir, filename)
        process.process_file(full_path, cnx)
        # os.remove(full_path)


# Close Postgres connection
cnx.close()